# AdventureWorks ETL Pipeline
## By Sonika Modur
This project is an end-to-end ETL solution built in Python that extracts, transforms, and loads data from multiple sources (MySQL, MongoDB, and CSV files) into a star schema data warehouse. The notebook demonstrates the complete workflow—from data extraction and transformation to loading—ensuring data quality and consistency throughout the process. For the full project including all dependencies, please visit my GitHub repository [here.](https://github.com/sonikarichamodur/adventureworks-etl-pipeline)

### Import Necessary Libraries

In [43]:
import os
import json
import numpy
import pandas as pd
import datetime
import certifi

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

In [44]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 1.4.7
Running PyMongo Version: 4.10.1


### Declare and Assign Connection Variables for MySQL Server and Databases

In [45]:
src_mysql_args = {
    "uid" : "root",
    "pwd" : "PASSWORD123!",
    "hostname" : "localhost",
    "dbname" : "adventureworks"
}

dst_mysql_args = {
    "uid" : "root",
    "pwd" : "PASSWORD123!",
    "hostname" : "localhost",
    "dbname" : "adventureworks_dw"
}

mongodb_args = {
    "user_name" : "",
    "password" : "password",
    "cluster_name" : "midtermproject",
    "cluster_subnet" : "",
    "cluster_location" : "local", # "local"
    "db_name" : "adventureworks_mongodb"
}

### Declare Functions for Getting Data From and Setting Data Into Databases (MySQL)

In [46]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    
    return dframe

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

### Declare Functions for Getting Data From and Setting Data Into Databases (MongoDB)

In [47]:
def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

### Create the AdventureWorks Data Warehouse

In [48]:
conn_str = f"mysql+pymysql://{dst_mysql_args['uid']}:{dst_mysql_args['pwd']}@{dst_mysql_args['hostname']}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_mysql_args['dbname']}`;"))
connection.execute(text(f"CREATE DATABASE `{dst_mysql_args['dbname']}`;"))
connection.execute(text(f"USE {dst_mysql_args['dbname']};"))

connection.close()

### ETL Process for MySQL

#### Write a SQL query for the product dimension

In [49]:
sql_products = """
SELECT
    p.ProductID,
    p.Name AS ProductName,
    p.ProductNumber,
    p.MakeFlag,
    p.FinishedGoodsFlag,
    p.Color,
    p.SafetyStockLevel,
    p.ReorderPoint,
    p.StandardCost,
    p.ListPrice,
    p.Size,
    p.SizeUnitMeasureCode,
    p.WeightUnitMeasureCode,
    p.Weight,
    p.DaysToManufacture,
    p.ProductLine,
    p.Class,
    p.Style,
    p.ProductSubcategoryID,
    p.ProductModelID,
    p.SellStartDate,
    p.SellEndDate,
    p.DiscontinuedDate,
    p.rowguid,
    p.ModifiedDate,
    psc.Name AS ProductSubcategoryName,
    pm.Name AS ProductModelName
    
FROM product p
LEFT JOIN productsubcategory psc ON p.ProductSubcategoryID = psc.ProductSubcategoryID
LEFT JOIN productmodel pm ON p.ProductModelID = pm.ProductModelID;
"""

#### Extract step: Create & populate product dimension table

In [50]:
df_products = get_sql_dataframe(sql_products, **src_mysql_args)
df_products.head(2)

,ProductID,ProductName,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate,ProductSubcategoryName,ProductModelName
0,1,Adjustable Race,AR-5381,b'\x00',b'\x00',None,1000,750,0.0,0.0,...,None,NaN,NaN,1998-06-01,NaT,None,b'\xb7\x15Bi\xf7\x08\rL\xac\xb1\xd74\xbaD\xc0\...,2004-03-11 10:01:36,None,None
1,2,Bearing Ball,BA-8327,b'\x00',b'\x00',None,1000,750,0.0,0.0,...,None,NaN,NaN,1998-06-01,NaT,None,b' <\xaeX:OIG\xa7\xd4\xd5h\x80l\xc57',2004-03-11 10:01:36,None,None


#### Transform step: drop, convert types, and reorder columns in product dimension table

In [51]:
# Drop redundant/irrelevant columns
df_products.drop(columns=[
    'DiscontinuedDate', # all null
    'rowguid', # useless
    'ModifiedDate', # all same value
    'ProductSubcategoryID', # redundant foreign key
    'ProductModelID' # redundant foreign key
], inplace=True)

# Reorder columns 
ordered_cols = [
    'ProductID',
    'ProductName',
    'ProductNumber',
    'MakeFlag',
    'FinishedGoodsFlag',
    'Color',
    'SafetyStockLevel',
    'ReorderPoint',
    'StandardCost',
    'ListPrice',
    'Size',
    'SizeUnitMeasureCode',
    'WeightUnitMeasureCode',
    'Weight',
    'DaysToManufacture',
    'ProductLine',
    'Class',
    'Style',
    'ProductSubcategoryName',
    'ProductModelName',
    'SellStartDate',
    'SellEndDate',
]

df_products = df_products[ordered_cols]


# Convert columns of type byte to type int 
def byte_to_int(val):
    if isinstance(val, bytes):
        return int.from_bytes(val, 'big')
    return val
df_products['MakeFlag'] = df_products['MakeFlag'].apply(byte_to_int)
df_products['FinishedGoodsFlag'] = df_products['FinishedGoodsFlag'].apply(byte_to_int)

# Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_products.insert(0, "ProductKey", range(1, df_products.shape[0] + 1))
df_products.head(2)

,ProductKey,ProductID,ProductName,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,...,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryName,ProductModelName,SellStartDate,SellEndDate
0,1,1,Adjustable Race,AR-5381,0,0,None,1000,750,0.0,...,None,NaN,0,None,None,None,None,None,1998-06-01,NaT
1,2,2,Bearing Ball,BA-8327,0,0,None,1000,750,0.0,...,None,NaN,0,None,None,None,None,None,1998-06-01,NaT


#### Load step: create and populate date dimension 

Execute the `Create_Populate_Dim_Date.sql` script to create and populate a date dimension table (`date_dim`) in the AdventureWorks data warehouse. The SQL file must be located in the working directory for this step. 

#### Load step: populate product dimension

In [52]:
db_operation = "insert"
set_dataframe(df_products, table_name='dim_products', pk_column='ProductKey',db_operation=db_operation, **dst_mysql_args)

#### Verify success of ETL operations for date dimension

In [53]:
# Retrieve and display date dimension table from adventureworks data warehouse
sql_dim_date = "SELECT * FROM dim_date;"
df_dim_date = get_sql_dataframe(sql_date, **dst_mysql_args)
df_dim_date.head(2)

,date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
1,20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


#### Verify success of ETL operations for product dimension

In [54]:
# Retrieve and display product dimension table from adventureworks data warehouse
sql_dim_products = "SELECT * FROM dim_products;"
df_dim_products = get_sql_dataframe(sql_dim_products, **dst_mysql_args)
df_dim_products.head(2)

,ProductKey,ProductID,ProductName,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,...,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryName,ProductModelName,SellStartDate,SellEndDate
0,1,1,Adjustable Race,AR-5381,0,0,None,1000,750,0.0,...,None,NaN,0,None,None,None,None,None,1998-06-01,NaT
1,2,2,Bearing Ball,BA-8327,0,0,None,1000,750,0.0,...,None,NaN,0,None,None,None,None,None,1998-06-01,NaT


### ETL Process for MongoDB - Populate MongoDB with Source Data

#### Write an SQL query for the employee dimension

Note: did not include rowguid in Employee table since it is irrelevant. Also did not include CurrentFlag in Employee table since its value is 1 for all employees. 

In [55]:
sql_dim_employees = """
SELECT
    e.EmployeeID,
    e.NationalIDNumber,
    e.ContactID,
    e.LoginID,
    e.ManagerID,
    e.Title AS EmployeeTitle,
    e.BirthDate,
    e.MaritalStatus,
    e.Gender,
    e.HireDate,
    e.SalariedFlag,
    e.VacationHours,
    e.SickLeaveHours,
    e.ModifiedDate AS EmployeeModifiedDate,

    c.FirstName,
    c.MiddleName,
    c.LastName,
    c.EmailAddress,
    c.Phone,
    c.ModifiedDate AS ContactModifiedDate,

    ea.AddressID,
    ea.ModifiedDate AS EmployeeAddressModifiedDate,

    a.AddressLine1,
    a.AddressLine2,
    a.City,
    a.PostalCode,
    a.ModifiedDate AS AddressModifiedDate,

    edh.DepartmentID,
    edh.ShiftID,
    edh.StartDate AS DeptStartDate,
    edh.EndDate AS DeptEndDate,
    edh.ModifiedDate AS DeptHistModifiedDate,

    d.Name AS DeptName,
    d.GroupName,
    d.ModifiedDate AS DeptModifiedDate,
    
    s.Name AS ShiftName,
    s.StartTime AS ShiftStartTime,
    s.EndTime AS ShiftEndTime,
    s.ModifiedDate AS ShiftModifiedDate,

    eph.RateChangeDate,
    eph.Rate,
    eph.PayFrequency,
    eph.ModifiedDate AS PayHistModifiedDate

FROM employee e
LEFT JOIN contact c
    ON e.ContactID = c.ContactID
LEFT JOIN employeeaddress ea
    ON e.EmployeeID = ea.EmployeeID
LEFT JOIN address a
    ON ea.AddressID = a.AddressID
LEFT JOIN employeedepartmenthistory edh
    ON e.EmployeeID = edh.EmployeeID
LEFT JOIN department d
    ON edh.DepartmentID = d.DepartmentID
LEFT JOIN shift s
    ON edh.ShiftID = s.ShiftID
LEFT JOIN employeepayhistory eph
    ON e.EmployeeID = eph.EmployeeID
"""

#### Extract step: get employee data from MySQL

In [56]:
df_employee = get_sql_dataframe(sql_dim_employees, **src_mysql_args)
df_employee.head(2)

,EmployeeID,NationalIDNumber,ContactID,LoginID,ManagerID,EmployeeTitle,BirthDate,MaritalStatus,Gender,HireDate,...,GroupName,DeptModifiedDate,ShiftName,ShiftStartTime,ShiftEndTime,ShiftModifiedDate,RateChangeDate,Rate,PayFrequency,PayHistModifiedDate
0,1,14417807,1209,adventure-works\guy1,16.0,Production Technician - WC60,1972-05-15,M,M,1996-07-31,...,Manufacturing,1998-06-01,Day,1900-01-01 07:00:00,1900-01-01 15:00:00,1998-06-01,1996-07-31,12.4500,1,2004-07-31
1,2,253022876,1030,adventure-works\kevin0,6.0,Marketing Assistant,1977-06-03,S,M,1997-02-26,...,Sales and Marketing,1998-06-01,Day,1900-01-01 07:00:00,1900-01-01 15:00:00,1998-06-01,1997-02-26,13.4615,2,2004-07-31


#### Validate dataframe columns for JSON conversion 
This was a troubleshooting step, since there was initially an error that some column(s) cannot be encoded into UTF-8 when converting the dataframe to JSON. Without being compatible for UTF-8 encoding, the dataframe cannot be converted into a JSON file. So, this code iterates through the columns to find which ones are problematic.

In [57]:
for col in df_employee.select_dtypes(include=['object']).columns:
    for value in df_employee[col]:
        if value is None:
            continue
        try:
            value.encode('utf-8')
        except Exception as e:
            print(f"Error in column '{col}' for value '{value}'")
            break # proceed to check next column 

Error in column 'SalariedFlag' for value 'b'\x00''


#### Transform step: convert byte values in SalariedFlag into Integers
The problematic column was found to be `SalariedFlag`, which had at least one value of type `byte`. This code iterates through the values in `SalariedFlag` and converts the byte types into integers so they are compatible with JSON.

In [58]:
def byte_to_int(val):
    if isinstance(val, bytes):
        return int.from_bytes(val, 'big')
    return val
df_employee['SalariedFlag'] = df_employee['SalariedFlag'].apply(byte_to_int)

#### Export dataframe resulting from SQL Query to JSON 

In [59]:
file = os.path.join(os.getcwd(), 'adventureworks_employee.json')
df_employee.to_json(file, orient='records')

#### Load step: Upload JSON data of employee dimension into MongoDB

In [60]:
client = get_mongo_client(**mongodb_args)
data_dir = os.getcwd()
set_mongo_collections(client, mongodb_args["db_name"], data_dir, {"employees":"adventureworks_employee.json"})      

### ETL Process for MongoDB - Create and Populate Employee Dimension Table

#### Extract step: get data from the source MongoDB collection into a DataFrame

In [61]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "employees"

df_employee = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_employee.head(2)

,EmployeeID,NationalIDNumber,ContactID,LoginID,ManagerID,EmployeeTitle,BirthDate,MaritalStatus,Gender,HireDate,...,GroupName,DeptModifiedDate,ShiftName,ShiftStartTime,ShiftEndTime,ShiftModifiedDate,RateChangeDate,Rate,PayFrequency,PayHistModifiedDate
0,1,14417807,1209,adventure-works\guy1,16.0,Production Technician - WC60,74736000000,M,M,838771200000,...,Manufacturing,896659200000,Day,-2208963600000,-2208934800000,896659200000,838771200000,12.4500,1,1091232000000
1,2,253022876,1030,adventure-works\kevin0,6.0,Marketing Assistant,234144000000,S,M,856915200000,...,Sales and Marketing,896659200000,Day,-2208963600000,-2208934800000,896659200000,856915200000,13.4615,2,1091232000000


#### Transform step: drop/rename/reorder columns in employees dimension 

In [62]:
# Drop redundant/irrelevant columns
df_employee.drop(columns=[
    'ContactID',
    'AddressID', 
    'DepartmentID', 
    'ShiftID', 
    'AddressModifiedDate',
    'DeptModifiedDate', 
    'ShiftModifiedDate'
], inplace=True)

df_employee = df_employee.rename(columns={
    'EmployeeAddressModifiedDate': 'AddressModifiedDate',
    'DeptHistModifiedDate': 'DeptModifiedDate',
})


ordered_cols = [
    "EmployeeID",
    "NationalIDNumber",
    "LoginID",
    "ManagerID",
    "EmployeeTitle",
    "BirthDate",
    "MaritalStatus",
    "Gender",
    "HireDate",
    "SalariedFlag",
    "VacationHours",
    "SickLeaveHours",
    "EmployeeModifiedDate",
    "FirstName",
    "MiddleName",
    "LastName",
    "EmailAddress",
    "Phone",
    "ContactModifiedDate",
    "AddressLine1",
    "AddressLine2",
    "City",
    "PostalCode",
    "AddressModifiedDate",
    "DeptName",
    "GroupName",
    "DeptStartDate",
    "DeptEndDate",
    "DeptModifiedDate",
    "ShiftName",
    "ShiftStartTime",
    "ShiftEndTime",
    "Rate",
    "RateChangeDate",
    "PayFrequency",
    "PayHistModifiedDate"
]

df_employee = df_employee[ordered_cols]

# 3. Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_employee.insert(0, "EmployeeKey", range(1, df_employee.shape[0] + 1))
df_employee.head(2)

,EmployeeKey,EmployeeID,NationalIDNumber,LoginID,ManagerID,EmployeeTitle,BirthDate,MaritalStatus,Gender,HireDate,...,DeptStartDate,DeptEndDate,DeptModifiedDate,ShiftName,ShiftStartTime,ShiftEndTime,Rate,RateChangeDate,PayFrequency,PayHistModifiedDate
0,1,1,14417807,adventure-works\guy1,16.0,Production Technician - WC60,74736000000,M,M,838771200000,...,838771200000,NaN,838684800000,Day,-2208963600000,-2208934800000,12.4500,838771200000,1,1091232000000
1,2,2,253022876,adventure-works\kevin0,6.0,Marketing Assistant,234144000000,S,M,856915200000,...,856915200000,NaN,856828800000,Day,-2208963600000,-2208934800000,13.4615,856915200000,2,1091232000000


#### Load step: load the transformed dataframe into AdventureWorks data warehouse by creating a new table

In [63]:
dataframe = df_employee
table_name = 'dim_employees'
primary_key = 'EmployeeKey'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **dst_mysql_args)

#### Validate that employee dimension table was created

In [64]:
sql_employees = "SELECT * FROM adventureworks_dw.dim_employees;"
df_dim_employees = get_sql_dataframe(sql_employees, **dst_mysql_args)
df_dim_employees.head(2)

,EmployeeKey,EmployeeID,NationalIDNumber,LoginID,ManagerID,EmployeeTitle,BirthDate,MaritalStatus,Gender,HireDate,...,DeptStartDate,DeptEndDate,DeptModifiedDate,ShiftName,ShiftStartTime,ShiftEndTime,Rate,RateChangeDate,PayFrequency,PayHistModifiedDate
0,1,1,14417807,adventure-works\guy1,16.0,Production Technician - WC60,74736000000,M,M,838771200000,...,838771200000,NaN,838684800000,Day,-2208963600000,-2208934800000,12.4500,838771200000,1,1091232000000
1,2,2,253022876,adventure-works\kevin0,6.0,Marketing Assistant,234144000000,S,M,856915200000,...,856915200000,NaN,856828800000,Day,-2208963600000,-2208934800000,13.4615,856915200000,2,1091232000000


### ETL Process for CSV file

#### Write an SQL query for the vendor dimension

In [65]:
sql_dim_vendors = """
SELECT
    v.VendorID,
    v.AccountNumber,
    v.Name AS VendorName,
    v.CreditRating,
    v.PreferredVendorStatus,
    v.ActiveFlag,
    v.PurchasingWebServiceURL,
    v.ModifiedDate AS VendorModifiedDate,

    va.AddressID,
    va.AddressTypeID,
    va.ModifiedDate AS VendorAddressModifiedDate,

    a.AddressLine1,
    a.AddressLine2,
    a.City,
    a.PostalCode,
    a.ModifiedDate AS AddressModifiedDate,

    at.Name AS AddressTypeName,

    vc.ContactID,
    vc.ContactTypeID,
    vc.ModifiedDate AS VendorContactModifiedDate,

    c.FirstName,
    c.MiddleName,
    c.LastName,
    c.EmailAddress,
    c.Phone,
    c.ModifiedDate AS ContactModifiedDate,

    ct.Name AS ContactTypeName

FROM vendor v
LEFT JOIN vendoraddress va 
       ON v.VendorID = va.VendorID
LEFT JOIN address a 
       ON va.AddressID = a.AddressID
LEFT JOIN addresstype at
       ON va.AddressTypeID = at.AddressTypeID
LEFT JOIN vendorcontact vc
       ON v.VendorID = vc.VendorID
LEFT JOIN contact c
       ON vc.ContactID = c.ContactID
LEFT JOIN contacttype ct
       ON vc.ContactTypeID = ct.ContactTypeID;
"""

#### Extract step: get vendor data from MySQL

In [66]:
df_vendor = get_sql_dataframe(sql_dim_vendors, **src_mysql_args)
df_vendor.head(2)

,VendorID,AccountNumber,VendorName,CreditRating,PreferredVendorStatus,ActiveFlag,PurchasingWebServiceURL,VendorModifiedDate,AddressID,AddressTypeID,...,ContactID,ContactTypeID,VendorContactModifiedDate,FirstName,MiddleName,LastName,EmailAddress,Phone,ContactModifiedDate,ContactTypeName
0,1,INTERNAT0001,International,1,b'\x01',b'\x01',None,2002-02-25,357,3,...,610,2,2002-02-25,Julia,None,Moseley,julia0@adventure-works.com,432-555-0100,2002-02-25,Assistant Sales Agent
1,2,ELECTRON0002,Electronic Bike Repair & Supplies,1,b'\x01',b'\x01',None,2002-02-17,335,3,...,678,2,2002-02-17,Sean,None,Purcell,sean5@adventure-works.com,130-555-0100,2002-02-17,Assistant Sales Agent


#### Transform step: drop, rename, and reorder columns in vendor dimension

In [67]:
# Drop redundant/irrelevant columns
df_vendor.drop(columns=[
    'PurchasingWebServiceURL',
    'AddressID',
    'AddressTypeID',
    'AddressModifiedDate',
    'ContactID',
    'ContactTypeID',
    'ContactModifiedDate'
], inplace=True)

# Rename columns 
df_vendor = df_vendor.rename(columns={
    'VendorContactModifiedDate': 'ContactModifiedDate',
    'VendorAddressModifiedDate': 'AddressModifiedDate',
    'ConactTypeName':'ContactType'
})

# Reorder columns
ordered_cols = [
    "VendorID",
    "AccountNumber",
    "VendorName",
    "CreditRating",
    "PreferredVendorStatus",
    "ActiveFlag",
    "VendorModifiedDate",
    "AddressLine1",
    "AddressLine2",
    "City",
    "PostalCode",
    "AddressModifiedDate",
    "FirstName",
    "MiddleName",
    "LastName",
    "EmailAddress",
    "Phone",
    "ContactTypeName",
    "ContactModifiedDate"
]

df_vendor = df_vendor[ordered_cols]

# 3. Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_vendor.insert(0, "VendorKey", range(1, df_vendor.shape[0] + 1))
df_vendor.head(2)

,VendorKey,VendorID,AccountNumber,VendorName,CreditRating,PreferredVendorStatus,ActiveFlag,VendorModifiedDate,AddressLine1,AddressLine2,City,PostalCode,AddressModifiedDate,FirstName,MiddleName,LastName,EmailAddress,Phone,ContactTypeName,ContactModifiedDate
0,1,1,INTERNAT0001,International,1,b'\x01',b'\x01',2002-02-25,683 Larch Ct.,None,Salt Lake City,84101,2002-02-25,Julia,None,Moseley,julia0@adventure-works.com,432-555-0100,Assistant Sales Agent,2002-02-25
1,2,2,ELECTRON0002,Electronic Bike Repair & Supplies,1,b'\x01',b'\x01',2002-02-17,8547 Catherine Way,None,Tacoma,98403,2002-02-17,Sean,None,Purcell,sean5@adventure-works.com,130-555-0100,Assistant Sales Agent,2002-02-17


#### Export data resulting from SQL query to CSV

In [68]:
df_vendor.to_csv('dim_vendors.csv', index=False)

#### Load step: Upload CSV data of vendor dimension into AdventureWorks data warehouse

In [69]:
df_dim_vendors = pd.read_csv('dim_vendors.csv')
dataframe = df_dim_vendors
table_name = 'dim_vendors'
primary_key = 'VendorKey'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **dst_mysql_args)

#### Validate that vendor dimension table was created

In [78]:
sql_vendors = "SELECT * FROM adventureworks_dw.dim_vendors;"
df_dim_vendors = get_sql_dataframe(sql_vendors, **dst_mysql_args)
df_dim_vendors.head(2)

,VendorKey,VendorID,AccountNumber,VendorName,CreditRating,PreferredVendorStatus,ActiveFlag,VendorModifiedDate,AddressLine1,AddressLine2,City,PostalCode,AddressModifiedDate,FirstName,MiddleName,LastName,EmailAddress,Phone,ContactTypeName,ContactModifiedDate
0,1,1,INTERNAT0001,International,1,b'\x01',b'\x01',2002-02-25,683 Larch Ct.,None,Salt Lake City,84101,2002-02-25,Julia,None,Moseley,julia0@adventure-works.com,432-555-0100,Assistant Sales Agent,2002-02-25
1,2,2,ELECTRON0002,Electronic Bike Repair & Supplies,1,b'\x01',b'\x01',2002-02-17,8547 Catherine Way,None,Tacoma,98403,2002-02-17,Sean,None,Purcell,sean5@adventure-works.com,130-555-0100,Assistant Sales Agent,2002-02-17


### Create, Populate, and Extract Fact Table (fact_purchaseorders)

#### Fill dataframes for the source tables needed to create fact_purchaseorders

In [71]:
# SELECT all columns from the adventureworks.purchaseorderdetail table to create the "df_po_detail" dataframe
sql_po_detail = "SELECT * FROM adventureworks.purchaseorderdetail;"
df_po_detail = get_sql_dataframe(sql_po_detail, **src_mysql_args)
# Display first two rows of the DataFrame to validate work
df_po_detail.head(2)

,PurchaseOrderID,PurchaseOrderDetailID,DueDate,OrderQty,ProductID,UnitPrice,LineTotal,ReceivedQty,RejectedQty,StockedQty,ModifiedDate
0,1,1,2001-05-31,4,1,50.26,201.04,3.0,0.0,3.0,2001-05-24
1,2,2,2001-05-31,3,359,45.12,135.36,3.0,0.0,3.0,2001-05-24


In [72]:
# SELECT all columns from the adventureworks.purchaseorderheader table to create the "df_po_header" dataframe
sql_po_header = "SELECT * FROM adventureworks.purchaseorderheader;"
df_po_header = get_sql_dataframe(sql_po_header, **src_mysql_args)
# Display first two rows of the DataFrame to validate work
df_po_header.head(2)

,PurchaseOrderID,RevisionNumber,Status,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate
0,1,0,4,244,83,3,2001-05-17,2001-05-26,201.0400,16.0832,5.0260,222.1492,2001-05-26
1,2,0,1,231,32,5,2001-05-17,2001-05-26,272.1015,21.7681,6.8025,300.6721,2001-05-26


#### Join the purchaseorderdetail and purchaseorderheader dataframes

In [73]:
df_fact_purchaseorders = pd.merge(df_po_detail, df_po_header, on='PurchaseOrderID', how='inner')
df_fact_purchaseorders.head(2)

,PurchaseOrderID,PurchaseOrderDetailID,DueDate,OrderQty,ProductID,UnitPrice,LineTotal,ReceivedQty,RejectedQty,StockedQty,...,EmployeeID,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate_y
0,1,1,2001-05-31,4,1,50.26,201.04,3.0,0.0,3.0,...,244,83,3,2001-05-17,2001-05-26,201.0400,16.0832,5.0260,222.1492,2001-05-26
1,2,2,2001-05-31,3,359,45.12,135.36,3.0,0.0,3.0,...,231,32,5,2001-05-17,2001-05-26,272.1015,21.7681,6.8025,300.6721,2001-05-26


#### Lookup the primary keys from the dimension tables
We need to establish foreign key relationships between the fact table and the dimension tables

First, get the Surrogate Primary Key and the Business Key from each of the Dimension tables using the get_dataframe() function.

In [75]:
# Select 'ProductKey' and 'ProductID' from adventureworks_dw.dim_products
dim_query = "SELECT ProductKey, ProductID FROM dim_products;"
df_dim_products = get_sql_dataframe(dim_query, **dst_mysql_args)
print(df_dim_products.head)

<bound method NDFrame.head of      ProductKey  ProductID
0             1          1
1             2          2
2             3          3
3             4          4
4             5        316
..          ...        ...
499         500        995
500         501        996
501         502        997
502         503        998
503         504        999

[504 rows x 2 columns]>


In [76]:
# Select 'EmployeeKey' and 'EmployeeID' from adventureworks_dw.dim_employees
dim_query = "SELECT EmployeeKey, EmployeeID FROM dim_employees;"
df_dim_employees = get_sql_dataframe(dim_query, **dst_mysql_args)
print(df_dim_employees.head)

<bound method NDFrame.head of      EmployeeKey  EmployeeID
0              1           1
1              2           2
2              3           3
3              4           4
4              5           4
..           ...         ...
329          330         286
330          331         287
331          332         288
332          333         289
333          334         290

[334 rows x 2 columns]>


In [77]:
# Select 'VendorKey' and 'VendorID' from adventureworks_dw.dim_vendors
dim_query = "SELECT VendorKey, VendorID FROM dim_vendors;"
df_dim_vendors = get_sql_dataframe(dim_query, **dst_mysql_args)
print(df_dim_vendors.head)

<bound method NDFrame.head of      VendorKey  VendorID
0            1         1
1            2         2
2            3         2
3            4         2
4            5         3
..         ...       ...
151        152       102
152        153       102
153        154       102
154        155       103
155        156       104

[156 rows x 2 columns]>


Now, look up the corresponding Surrogate Primary Key values in the Dimension tables using the Business Keys

In [79]:
# Modify 'df_fact_purchaseorders' by merging it with 'df_dim_products' on the 'ProductID' column
df_fact_purchaseorders = df_fact_purchaseorders.merge(df_dim_products, on='ProductID', how='left')
# Drop the 'ProductID' column
df_fact_purchaseorders.drop(columns=['ProductID'], inplace = True)

df_fact_purchaseorders.head(2)

,PurchaseOrderID,PurchaseOrderDetailID,DueDate,OrderQty,UnitPrice,LineTotal,ReceivedQty,RejectedQty,StockedQty,ModifiedDate_x,...,VendorID,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate_y,ProductKey
0,1,1,2001-05-31,4,50.26,201.04,3.0,0.0,3.0,2001-05-24,...,83,3,2001-05-17,2001-05-26,201.0400,16.0832,5.0260,222.1492,2001-05-26,1
1,2,2,2001-05-31,3,45.12,135.36,3.0,0.0,3.0,2001-05-24,...,32,5,2001-05-17,2001-05-26,272.1015,21.7681,6.8025,300.6721,2001-05-26,38


In [80]:
# Modify 'df_fact_purchaseorders' by merging it with 'df_dim_employees' on the 'EmployeeID' column
df_fact_purchaseorders = df_fact_purchaseorders.merge(df_dim_employees, on='EmployeeID', how='left')
# Drop the 'EmployeeID' column
df_fact_purchaseorders.drop(columns=['EmployeeID'], inplace = True)

df_fact_purchaseorders.head(2)

,PurchaseOrderID,PurchaseOrderDetailID,DueDate,OrderQty,UnitPrice,LineTotal,ReceivedQty,RejectedQty,StockedQty,ModifiedDate_x,...,ShipMethodID,OrderDate,ShipDate,SubTotal,TaxAmt,Freight,TotalDue,ModifiedDate_y,ProductKey,EmployeeKey
0,1,1,2001-05-31,4,50.26,201.04,3.0,0.0,3.0,2001-05-24,...,3,2001-05-17,2001-05-26,201.0400,16.0832,5.0260,222.1492,2001-05-26,1,280
1,2,2,2001-05-31,3,45.12,135.36,3.0,0.0,3.0,2001-05-24,...,5,2001-05-17,2001-05-26,272.1015,21.7681,6.8025,300.6721,2001-05-26,38,267


In [81]:
# Modify 'df_fact_purchaseorders' by merging it with 'df_dim_vendors' on the 'VendorID' column
df_fact_purchaseorders = df_fact_purchaseorders.merge(df_dim_vendors, on='VendorID', how='left')
# Drop the 'VendorID' column
df_fact_purchaseorders.drop(columns=['VendorID'], inplace = True)

df_fact_purchaseorders.head(2)

,PurchaseOrderID,PurchaseOrderDetailID,DueDate,OrderQty,UnitPrice,LineTotal,ReceivedQty,RejectedQty,StockedQty,ModifiedDate_x,...,City,PostalCode,AddressModifiedDate,FirstName,MiddleName,LastName,EmailAddress,Phone,ContactTypeName,ContactModifiedDate
0,1,1,2001-05-31,4,50.26,201.04,3.0,0.0,3.0,2001-05-24,...,Santa Cruz,95062,2001-05-26,Lori,None,Oviatt,lori3@adventure-works.com,529-555-0100,Sales Manager,2001-05-26
1,2,2,2001-05-31,3,45.12,135.36,3.0,0.0,3.0,2001-05-24,...,Lynnwood,98036,2001-05-26,Jonathan,None,Moeller,jonathan0@adventure-works.com,859-555-0100,Sales Associate,2001-05-26


#### Lookup the DateKeys from the Date Dimension Table

In [83]:
# Find the date columns in the fact table
df_fact_purchaseorders.info()
# columns that will join to date dimension: DueDate, OrderDate, ShipDate

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18611 entries, 0 to 18610
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   PurchaseOrderID        18611 non-null  int64         
 1   PurchaseOrderDetailID  18611 non-null  int64         
 2   DueDate                18611 non-null  datetime64[ns]
 3   OrderQty               18611 non-null  int64         
 4   UnitPrice              18611 non-null  float64       
 5   LineTotal              18611 non-null  float64       
 6   ReceivedQty            18611 non-null  float64       
 7   RejectedQty            18611 non-null  float64       
 8   StockedQty             18611 non-null  float64       
 9   ModifiedDate_x         18611 non-null  datetime64[ns]
 10  RevisionNumber         18611 non-null  int64         
 11  Status                 18611 non-null  int64         
 12  ShipMethodID           18611 non-null  int64         
 13  O

In [85]:
sql_dim_date = "SELECT date_key, full_date FROM adventureworks_dw.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **dst_mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02


In [87]:
# Lookup the Surrogate Primary Key (date_key) that Corresponds to the "DueDate" Column.
df_dim_due_date = df_dim_date.rename(columns={"date_key" : "DueDateKey", "full_date" : "DueDate"})
df_fact_purchaseorders.DueDate = df_fact_purchaseorders.DueDate.astype('datetime64[ns]').dt.date
df_fact_purchaseorders = pd.merge(df_fact_purchaseorders, df_dim_due_date, on='DueDate', how='left')
df_fact_purchaseorders.drop(['DueDate'], axis=1, inplace=True)
df_fact_purchaseorders.head(2)

,PurchaseOrderID,PurchaseOrderDetailID,OrderQty,UnitPrice,LineTotal,ReceivedQty,RejectedQty,StockedQty,ModifiedDate_x,RevisionNumber,...,PostalCode,AddressModifiedDate,FirstName,MiddleName,LastName,EmailAddress,Phone,ContactTypeName,ContactModifiedDate,DueDateKey
0,1,1,4,50.26,201.04,3.0,0.0,3.0,2001-05-24,0,...,95062,2001-05-26,Lori,None,Oviatt,lori3@adventure-works.com,529-555-0100,Sales Manager,2001-05-26,20010531
1,2,2,3,45.12,135.36,3.0,0.0,3.0,2001-05-24,0,...,98036,2001-05-26,Jonathan,None,Moeller,jonathan0@adventure-works.com,859-555-0100,Sales Associate,2001-05-26,20010531


In [88]:
# Lookup the Surrogate Primary Key (date_key) that Corresponds to the "OrderDate" Column.
df_dim_order_date = df_dim_date.rename(columns={"date_key" : "OrderDateKey", "full_date" : "OrderDate"})
df_fact_purchaseorders.OrderDate = df_fact_purchaseorders.OrderDate.astype('datetime64[ns]').dt.date
df_fact_purchaseorders = pd.merge(df_fact_purchaseorders, df_dim_order_date, on='OrderDate', how='left')
df_fact_purchaseorders.drop(['OrderDate'], axis=1, inplace=True)
df_fact_purchaseorders.head(2)

,PurchaseOrderID,PurchaseOrderDetailID,OrderQty,UnitPrice,LineTotal,ReceivedQty,RejectedQty,StockedQty,ModifiedDate_x,RevisionNumber,...,AddressModifiedDate,FirstName,MiddleName,LastName,EmailAddress,Phone,ContactTypeName,ContactModifiedDate,DueDateKey,OrderDateKey
0,1,1,4,50.26,201.04,3.0,0.0,3.0,2001-05-24,0,...,2001-05-26,Lori,None,Oviatt,lori3@adventure-works.com,529-555-0100,Sales Manager,2001-05-26,20010531,20010517
1,2,2,3,45.12,135.36,3.0,0.0,3.0,2001-05-24,0,...,2001-05-26,Jonathan,None,Moeller,jonathan0@adventure-works.com,859-555-0100,Sales Associate,2001-05-26,20010531,20010517


In [89]:
# Lookup the Surrogate Primary Key (date_key) that Corresponds to the "ShipDate" Column.
df_dim_ship_date = df_dim_date.rename(columns={"date_key" : "ShipDateKey", "full_date" : "ShipDate"})
df_fact_purchaseorders.ShipDate = df_fact_purchaseorders.ShipDate.astype('datetime64[ns]').dt.date
df_fact_purchaseorders = pd.merge(df_fact_purchaseorders, df_dim_ship_date, on='ShipDate', how='left')
df_fact_purchaseorders.drop(['ShipDate'], axis=1, inplace=True)
df_fact_purchaseorders.head(2)

,PurchaseOrderID,PurchaseOrderDetailID,OrderQty,UnitPrice,LineTotal,ReceivedQty,RejectedQty,StockedQty,ModifiedDate_x,RevisionNumber,...,FirstName,MiddleName,LastName,EmailAddress,Phone,ContactTypeName,ContactModifiedDate,DueDateKey,OrderDateKey,ShipDateKey
0,1,1,4,50.26,201.04,3.0,0.0,3.0,2001-05-24,0,...,Lori,None,Oviatt,lori3@adventure-works.com,529-555-0100,Sales Manager,2001-05-26,20010531,20010517,20010526
1,2,2,3,45.12,135.36,3.0,0.0,3.0,2001-05-24,0,...,Jonathan,None,Moeller,jonathan0@adventure-works.com,859-555-0100,Sales Associate,2001-05-26,20010531,20010517,20010526


#### Perform any Additional Transformations

In [98]:
# Rename ModifiedDate_x and ModifiedDate_y
df_fact_purchaseorders.rename(columns={
    'ModifiedDate_x': 'PurchaseOrderDetailModifiedDate',
    'ModifiedDate_y': 'PurchaseOrderHeaderModifiedDate'
}, inplace=True)


# Drop columns that belong in dimension tables
drop_cols = [
    'AccountNumber',
    'VendorName',
    'CreditRating',
    'PreferredVendorStatus',
    'ActiveFlag',
    'VendorModifiedDate',
    'AddressLine1',
    'AddressLine2',
    'City',
    'PostalCode',
    'AddressModifiedDate',
    'FirstName',
    'MiddleName',
    'LastName',
    'EmailAddress',
    'Phone',
    'ContactTypeName',
    'ContactModifiedDate'
]


df_fact_purchaseorders.drop(columns=drop_cols)

# Reorder columns in fact table
ordered_cols = [
    'PurchaseOrderID',
    'PurchaseOrderDetailID',
    'ProductKey',
    'EmployeeKey',
    'VendorKey',
    'DueDateKey',
    'OrderDateKey',
    'ShipDateKey',
    'OrderQty',
    'UnitPrice',
    'LineTotal',
    'ReceivedQty',
    'RejectedQty',
    'StockedQty',
    'RevisionNumber',
    'Status',
    'ShipMethodID',
    'SubTotal',
    'TaxAmt',
    'Freight',
    'TotalDue',
    'PurchaseOrderDetailModifiedDate',
    'PurchaseOrderHeaderModifiedDate'
]

df_fact_purchaseorders = df_fact_purchaseorders[ordered_cols]

# Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_fact_purchaseorders.insert(0, 'FactPurchaseOrderKey', range(1, len(df_fact_purchaseorders) + 1))

df_fact_purchaseorders.head(2)

,FactPurchaseOrderKey,PurchaseOrderID,PurchaseOrderDetailID,ProductKey,EmployeeKey,VendorKey,DueDateKey,OrderDateKey,ShipDateKey,OrderQty,...,StockedQty,RevisionNumber,Status,ShipMethodID,SubTotal,TaxAmt,Freight,TotalDue,PurchaseOrderDetailModifiedDate,PurchaseOrderHeaderModifiedDate
0,1,1,1,1,280,125,20010531,20010517,20010526,4,...,3.0,0,4,3,201.0400,16.0832,5.0260,222.1492,2001-05-24,2001-05-26
1,2,2,2,38,267,50,20010531,20010517,20010526,3,...,3.0,0,1,5,272.1015,21.7681,6.8025,300.6721,2001-05-24,2001-05-26


#### Write dataframe back to database

In [102]:
dataframe = df_fact_purchaseorders
table_name = "fact_purchaseorders"
primary_key = "FactPurchaseOrderKey"
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **dst_mysql_args)

#### Demonstrate that the New Data Warehouse Exists and Contains the Correct Data

In [107]:
sql_test = """
SELECT 
    v.VendorName,
    p.ProductName,
    SUM(f.OrderQty) AS total_quantity,
    SUM(f.LineTotal) AS total_line_total,
    SUM(f.UnitPrice) AS total_unit_price
FROM {0}.fact_purchaseorders f
JOIN {0}.dim_vendors v
    ON f.VendorKey = v.VendorKey
JOIN {0}.dim_products p
    ON f.ProductKey = p.ProductKey
GROUP BY 
    v.VendorName,
    p.ProductName
ORDER BY 
    v.VendorName,
    p.ProductName;
""".format("adventureworks_dw")

df_test = get_sql_dataframe(sql_test, **dst_mysql_args)
df_test.head()

,VendorName,ProductName,total_quantity,total_line_total,total_unit_price
0,Advanced Bicycles,Thin-Jam Hex Nut 1,117.0,5832.9180,1944.3060
1,Advanced Bicycles,Thin-Jam Hex Nut 10,189.0,8614.7145,2871.5715
2,Advanced Bicycles,Thin-Jam Hex Nut 11,108.0,4015.4940,1338.4980
3,Advanced Bicycles,Thin-Jam Hex Nut 12,108.0,4477.0320,1492.3440
4,Advanced Bicycles,Thin-Jam Hex Nut 13,180.0,7070.4900,2356.8300
